Henry Ho 304723723

# League Rank Predictor

Uses Riot Developer's api V3 (https://developer.riotgames.com/) and seeded data to predict the "highest achieved season tier" for each player using the data from matches as features.

In [1]:
import pandas as pd
import time

## Read API key

### Get your own API key from https://developer.riotgames.com/ and put it in api_key.txt

In [2]:
# Reads the api key from a .txt file "api_key.txt"

file_path = "api_key.txt"

api_key = ""

with open(file_path) as f:
    api_key = f.readline()

## Function for searching by summoner name

In [3]:
import json
import requests

# Searches name in api and returns the player's account id
def get_account_id(name):
    
    URL = "https://na1.api.riotgames.com/lol/summoner/v3/summoners/"
    
    URI = "{}by-name/{}?api_key={}".format(URL, name, api_key)
    response = requests.get(URI)
    account_id = response.json()['accountId']
    return account_id

## Function for getting the match_ids of the last 20 matches played with summoner name

In [4]:
# Searches name in api and returns a list of match ids of the last 20 matches played
def get_match_ids(name):
    
    match_ids = []
    
    URL = "https://na1.api.riotgames.com/lol/match/v3/matchlists/"
    
    account_id = get_account_id(name)
    
    URI = "{}by-account/{}/recent?api_key={}".format(URL, account_id, api_key)
    response = requests.get(URI)
    
    for matches in response.json()['matches']:
        match_id = matches['gameId']
        match_ids.append(match_id)
    
    return match_ids

## Function for getting dataframe of player's stats with the summoner name

In [5]:
# Searches name in api and returns a dataframe of the player's stats
def get_stats(name):
    
    stats_df = pd.DataFrame(columns = ['win', 'role', 'kills', 'deaths', 'assists', 'gameDuration', 'goldEarned', 'totalMinionsKilled', 'damageDealtToObjectives', 'totalDamageDealt', 'totalDamageTaken', 'wardsPlaced', 'highestAchievedSeasonTier'])
    matches = []
    participant_ids = []
    
    URL = "https://na1.api.riotgames.com/lol/match/v3/"
    
    account_id = get_account_id(name)
    match_ids = get_match_ids(name)
    
    # Gets participant id of the match from all 20 matches
    for match_id in match_ids:
        URI = "{}matches/{}?api_key={}".format(URL, match_id, api_key)
        response = requests.get(URI)
        
        matches.append(response.json())
        
        for participant in response.json()['participantIdentities']:
            if participant['player']['accountId'] == account_id:
                participant_id = participant['participantId']
                participant_ids.append(participant_id)
                
        # .2 second delay for rate api rate limits
        time.sleep(.2)
        
    # Gets player's stats and inserts it into a dataframe
    for match in matches:
        for participant in match['participants']:
            if participant['participantId'] == participant_ids[0] and match['gameMode'] != 'ARAM':
                stats_df = stats_df.append({'win': participant['stats']['win'],
                                            'role': participant['timeline']['role'],
                                            'kills': participant['stats']['kills'],
                                            'deaths': participant['stats']['deaths'],
                                            'assists': participant['stats']['assists'],
                                            'gameDuration': match['gameDuration'],
                                            'goldEarned': participant['stats']['goldEarned'],
                                            'totalMinionsKilled': participant['stats']['totalMinionsKilled'],
                                            'damageDealtToObjectives': participant['stats']['damageDealtToObjectives'],
                                            'totalDamageDealt': participant['stats']['totalDamageDealt'],
                                            'totalDamageTaken': participant['stats']['totalDamageTaken'],
                                            'wardsPlaced': participant['stats']['wardsPlaced'],
                                            'highestAchievedSeasonTier': participant['highestAchievedSeasonTier']}, ignore_index=True)

                participant_ids.pop(0)
                break
                
    return stats_df
            

In [6]:
name = "Imaqtpie"
print("name: {}\n".format(name))

account_id = get_account_id(name)
print("account id: {}\n".format(account_id))

match_ids = get_match_ids(name)
print("match ids: {}\n".format(match_ids))

stats = get_stats(name)
print("stats: ")
print(stats)
print("\n")

# TODO: GET THE MEAN OF THE STATS OF THE LAST 20 MATCHES

# TODO: TEST THE MEAN OF THE STATS ON THE TRAINED DATASET

name: Imaqtpie

account id: 32639237

match ids: [2648723830, 2638815666, 2634541288, 2634537914, 2634528085, 2634534978, 2629001744, 2628979135, 2628984310, 2628971679, 2628769295, 2628817078, 2628814705, 2628802983, 2628762055, 2628810855, 2623341442, 2623239133, 2623228110, 2623236486]

stats: 
      win         role kills deaths assists gameDuration goldEarned  \
0   False    DUO_CARRY     8      9       9         1802      12823   
1   False         SOLO     2      4      12         1870      11271   
2   False         SOLO     1      7       2          941       4445   
3    True         SOLO    10      3       9         1336      10985   
4   False    DUO_CARRY     5     10       1         1645      10178   
5    True    DUO_CARRY     7      2      10         2006      14511   
6    True  DUO_SUPPORT     0      0      11         1217       7488   
7    True    DUO_CARRY    12      4       6         1943      15454   
8   False         NONE     3      6       6         1876      